In [1]:
import os
import csv
import json
import shutil
from glob import glob
from datetime import datetime

from my_functions import *
# imports functions: mylistdir, make_storage_directory, and get_date_list

***
## Whole sheet inputs

In [2]:

# Local file storage
local_save_path = '/Users/maggie/Desktop/HDstorage'

# Final location for compressed file storage
dest_for_compressed = '/Volumes/SAMSUNG-4/'

start_end_file = 'start_end_dates.json'

# Path to home files (external HD)
from_path = '/Volumes/TOSHIBA-18/H6-black/'

In [3]:


home, c = os.path.basename(from_path.strip('/')).split('-')
color = c[0].upper()
hub_paths = glob(os.path.join(from_path, f'{color}S*'))
    
all_days = get_date_list(read_file=start_end_file, H_num=home)
print(f'Number of days in home {home}: {len(all_days)}')
print('Hub paths:')
for hub in hub_paths:
    print(hub)

Number of days in home H6: 38
Hub paths:
/Volumes/TOSHIBA-18/H6-black/BS2
/Volumes/TOSHIBA-18/H6-black/BS3
/Volumes/TOSHIBA-18/H6-black/BS4
/Volumes/TOSHIBA-18/H6-black/BS5


***
## A. Dark Image Lists
Final csv files should be in form:  
`HOME_DARKIMAGES.zip/HOME_HUB_DARKIMAGES/date_hub_home_dark-images.csv`

1. Re-write the (date-appropriate) json files (stored in `hub_path`) to `.csv` files stored in the `local_save_path`. 
2. Compress the final folder to `dest_for_compressed`.

### 1. Rewrite Dark image JSON files to csvs and rename storage directories

In [10]:
save_loc = make_storage_directory(os.path.join(local_save_path, home, f'{home}_DARKIMAGES'))


for hub_path in hub_paths:
    hub = os.path.basename(hub_path.strip('/'))
    save_dirname = f'{home}_{hub}_DARKIMAGES'
#     print(os.path.join(save_loc, save_dirname))
    save_hub_dir = make_storage_directory(os.path.join(save_loc, save_dirname))
    all_files = glob(os.path.join(hub_path, '*-img-pkl', 'black_image_dicts', '*.json'))
    files_to_process = [f for f in all_files if os.path.basename(f).strip('_dark_images.json') in all_days]
    print(hub, len(files_to_process))
    for json_file in files_to_process:
        day = os.path.basename(json_file).strip('_dark_images.json')
        with open(json_file) as json_file:
            data = json.load(json_file)
            times = []
            for key in data:
                times.extend(data[key])
        timestamps = sorted([datetime.strptime(f'{day} {time}', '%Y-%m-%d %H%M%S') for time in times])
        timestamps = [[ts] for ts in timestamps]

        csv_file = os.path.join(save_hub_dir, f'{day}_{hub}_{home}_darkimages.csv')
        write_file = open(csv_file, 'w+')
        with write_file:
            csv_writer = csv.writer(write_file)
            csv_writer.writerows(timestamps)

BS2 38
BS3 38
BS4 38
BS5 38


### 2. Compress dark image files to harddrive

In [11]:
to_path = make_storage_directory(os.path.join(dest_for_compressed, home))

to_compress = os.path.join(local_save_path, home, f'{home}_DARKIMAGES')
destination = os.path.join(to_path, os.path.basename(to_compress.strip('/')))

print(f'from: {to_compress} to: {destination}')
shutil.make_archive(destination, 'zip', to_compress)

from: /Users/maggie/Desktop/HDstorage/H6/H6_DARKIMAGES to: /Volumes/SAMSUNG-4/H6/H6_DARKIMAGES


'/Volumes/SAMSUNG-4/H6/H6_DARKIMAGES.zip'

***
## B. Images

Final images should be in form:  
`from_path/hub/HOME_HUB_IMAGES/date/minute/date_time_hub_home.png`. <br>
where `date`, `minute`, and `time` are `YYYY-mm-dd`, `hhmm`, and `hhmmss`, respectively.

Zip files should be stored on `dest_for_compressed` under `HOME_HUB_IMAGES.zip`

1. Create new day-wise folder of copied images to include on `from_path` by moving ones NOT to compress to archived folder.
2. Compress images to external `dest_for_compressed`.


### 1. Move image day-files not part of the set to an archive folder

In [4]:
for hub_path in hub_paths:
    hub = os.path.basename(hub_path.strip('/'))
    day_files = glob(os.path.join(hub_path, 'img-downsized-32', '2019-*'))
    move_files = [f for f in day_files if os.path.basename(f) not in all_days]
    print(hub, len(day_files), len(move_files))
    target_dir = make_storage_directory(os.path.join(hub_path, 'img-downsized_archive'))
    for f_to_move in move_files:
        print(f_to_move, target_dir)
#         shutil.move(f_to_move, target_dir)

BS2 38 0
BS3 38 0
BS4 38 0
BS5 38 0


### 2. Compress image files to harddrive

In [5]:

for hub_path in hub_paths:
    img_path = os.path.join(hub_path, 'img-downsized-32')
    hub = os.path.basename(hub_path)
    print(f'Days to compress on hub {hub}: {len(glob(os.path.join(img_path, "2019-*")))}')
    to_compress = img_path
    destination = os.path.join(dest_for_compressed, home, f'{home}_{hub}_IMAGES')
    print(f'from: {to_compress} to: {destination}')
    shutil.make_archive(destination, 'zip', to_compress)
    print('done compressing ', destination)


Days to compress on hub BS2: 38
from: /Volumes/TOSHIBA-18/H6-black/BS2/img-downsized-32 to: /Volumes/SAMSUNG-4/H6/H6_BS2_IMAGES
done compressing  /Volumes/SAMSUNG-4/H6/H6_BS2_IMAGES
Days to compress on hub BS3: 38
from: /Volumes/TOSHIBA-18/H6-black/BS3/img-downsized-32 to: /Volumes/SAMSUNG-4/H6/H6_BS3_IMAGES
done compressing  /Volumes/SAMSUNG-4/H6/H6_BS3_IMAGES
Days to compress on hub BS4: 38
from: /Volumes/TOSHIBA-18/H6-black/BS4/img-downsized-32 to: /Volumes/SAMSUNG-4/H6/H6_BS4_IMAGES
done compressing  /Volumes/SAMSUNG-4/H6/H6_BS4_IMAGES
Days to compress on hub BS5: 38
from: /Volumes/TOSHIBA-18/H6-black/BS5/img-downsized-32 to: /Volumes/SAMSUNG-4/H6/H6_BS5_IMAGES
done compressing  /Volumes/SAMSUNG-4/H6/H6_BS5_IMAGES


***
## C. Audio
Final audio files, stored as csv, should be in form:  
`HOME_AUDIO.zip/HOME_HUB_AUDIO/date/minute/date_time_hub_home_.csv` <br>
where `date`, `minute`, and `time` are `YYYY-mm-dd`, `hhmm`, and `hhmmss`, respectively.

1. Copy the day-wise audio files (stored in `hub_path`) to same structured files stored in the `local_save_path`. 
2. Compress the final folder with all hubs to `dest_for_compressed`.



### 1. Combine hub-folders for audio into one folder

In [6]:

for hub_path in hub_paths:
    print(hub_path)
    hub = os.path.basename(hub_path.strip('/'))
    day_dirs = glob(os.path.join(hub_path, 'processed_audio', 'audio_csv', '2019-*'))
    copy_dirs = [d for d in day_dirs if os.path.basename(d) in all_days]
    print(hub, len(day_dirs), len(copy_dirs))
    target_dir = make_storage_directory(os.path.join(local_save_path, home, f'{home}_AUDIO', f'{home}_{hub}_AUDIO'))
    for c_dir in copy_dirs:
        date = os.path.basename(c_dir)
        fname = os.path.join(target_dir, date)
        print(f'from: {c_dir} to: {fname}')
        shutil.copytree(c_dir, fname)

/Volumes/TOSHIBA-18/H6-black/BS2
BS2 38 38
from: /Volumes/TOSHIBA-18/H6-black/BS2/processed_audio/audio_csv/2019-10-12 to: /Users/maggie/Desktop/HDstorage/H6/H6_AUDIO/H6_BS2_AUDIO/2019-10-12
from: /Volumes/TOSHIBA-18/H6-black/BS2/processed_audio/audio_csv/2019-10-13 to: /Users/maggie/Desktop/HDstorage/H6/H6_AUDIO/H6_BS2_AUDIO/2019-10-13
from: /Volumes/TOSHIBA-18/H6-black/BS2/processed_audio/audio_csv/2019-10-14 to: /Users/maggie/Desktop/HDstorage/H6/H6_AUDIO/H6_BS2_AUDIO/2019-10-14
from: /Volumes/TOSHIBA-18/H6-black/BS2/processed_audio/audio_csv/2019-10-15 to: /Users/maggie/Desktop/HDstorage/H6/H6_AUDIO/H6_BS2_AUDIO/2019-10-15
from: /Volumes/TOSHIBA-18/H6-black/BS2/processed_audio/audio_csv/2019-10-16 to: /Users/maggie/Desktop/HDstorage/H6/H6_AUDIO/H6_BS2_AUDIO/2019-10-16
from: /Volumes/TOSHIBA-18/H6-black/BS2/processed_audio/audio_csv/2019-10-17 to: /Users/maggie/Desktop/HDstorage/H6/H6_AUDIO/H6_BS2_AUDIO/2019-10-17
from: /Volumes/TOSHIBA-18/H6-black/BS2/processed_audio/audio_csv/2019

### 2. Compress audio files to harddrive

In [7]:

to_compress = os.path.join(local_save_path, home, f'{home}_AUDIO')
destination = os.path.join(dest_for_compressed, home, os.path.basename(to_compress.strip('/')))

print(f'from: {to_compress} to: {destination}')
shutil.make_archive(destination, 'zip', to_compress)


# to_compress = os.path.join(directory_path, home, f'{home}_ENVIRONMENTAL')
# destination = os.path.join(dest_for_compressed, home, os.path.basename(to_compress.strip('/')))

# print(f'from: {to_compress} to: {destination}')
# shutil.make_archive(destination, 'zip', to_compress)


from: /Users/maggie/Desktop/HDstorage/H6/H6_AUDIO to: /Volumes/SAMSUNG-4/H6/H6_AUDIO


'/Volumes/SAMSUNG-4/H6/H6_AUDIO.zip'

***
## D. Environmental
1. Copy over to 'local_save_path`. Also need to change filename to match conventions
2. Compress the final folder with all hubs to `dest_for_compressed`.

### 1. Copy while changing names to match other conventions

In [8]:


for hub_path in hub_paths:
    hub = os.path.basename(hub_path.strip('/'))
    day_files = glob(os.path.join(hub_path, 'processed_env', 'CSV-raw', '*.csv'))
    copy_files = [f for f in day_files if os.path.basename(f).split('_')[2].rstrip('.csv') in all_days]
    print(hub, len(day_files), len(copy_files))
    target_dir = make_storage_directory(os.path.join(local_save_path, home, f'{home}_ENVIRONMENTAL', f'{home}_{hub}_ENV'))
    print(target_dir)
    for f in copy_files:
        date = os.path.basename(f).split('_')[2].rstrip('.csv')
        fname = os.path.join(target_dir, f'{date}_{hub}_{home}_env.csv')
        shutil.copy(f, fname)


BS2 64 38
/Users/maggie/Desktop/HDstorage/H6/H6_ENVIRONMENTAL/H6_BS2_ENV
BS3 64 38
/Users/maggie/Desktop/HDstorage/H6/H6_ENVIRONMENTAL/H6_BS3_ENV
BS4 64 38
/Users/maggie/Desktop/HDstorage/H6/H6_ENVIRONMENTAL/H6_BS4_ENV
BS5 64 38
/Users/maggie/Desktop/HDstorage/H6/H6_ENVIRONMENTAL/H6_BS5_ENV


### 2. Compress env to harddrive

In [9]:
to_compress = os.path.join(local_save_path, home, f'{home}_ENVIRONMENTAL')
destination = os.path.join(dest_for_compressed, home, os.path.basename(to_compress.strip('/')))

print(f'from: {to_compress} to: {destination}')
shutil.make_archive(destination, 'zip', to_compress)

from: /Users/maggie/Desktop/HDstorage/H6/H6_ENVIRONMENTAL to: /Volumes/SAMSUNG-4/H6/H6_ENVIRONMENTAL


'/Volumes/SAMSUNG-4/H6/H6_ENVIRONMENTAL.zip'

***
## E. Ground Truth
Final ground truth files, stored as csv, should be in form:  
`HOME_GROUNDTRUTH.zip/date/date_home_groundtruth.csv` <br>

1. Copy the day-wise ground truth files (stored in `hub_path`) to `local_save_path`. 
2. Compress the final folder with all hubs to `dest_for_compressed`.

### 1. Copy over relevant days

In [12]:
day_files = glob(os.path.join(from_path, 'GroundTruth/GroundTruth', '*_groundtruth.csv'))

copy_files = [f for f in day_files if os.path.basename(f).split('_')[0] in all_days]
save_loc = make_storage_directory(os.path.join(local_save_path, home, f'{home}_GROUNDTRUTH'))
print(len(copy_files))

for f in copy_files:
    dest_fname = os.path.join(save_loc, os.path.basename(f))
    print(f, dest_fname)
    shutil.copy(f, dest_fname)


38
/Volumes/TOSHIBA-18/H6-black/GroundTruth/GroundTruth/2019-10-12_H6_groundtruth.csv /Users/maggie/Desktop/HDstorage/H6/H6_GROUNDTRUTH/2019-10-12_H6_groundtruth.csv
/Volumes/TOSHIBA-18/H6-black/GroundTruth/GroundTruth/2019-10-13_H6_groundtruth.csv /Users/maggie/Desktop/HDstorage/H6/H6_GROUNDTRUTH/2019-10-13_H6_groundtruth.csv
/Volumes/TOSHIBA-18/H6-black/GroundTruth/GroundTruth/2019-10-14_H6_groundtruth.csv /Users/maggie/Desktop/HDstorage/H6/H6_GROUNDTRUTH/2019-10-14_H6_groundtruth.csv
/Volumes/TOSHIBA-18/H6-black/GroundTruth/GroundTruth/2019-10-15_H6_groundtruth.csv /Users/maggie/Desktop/HDstorage/H6/H6_GROUNDTRUTH/2019-10-15_H6_groundtruth.csv
/Volumes/TOSHIBA-18/H6-black/GroundTruth/GroundTruth/2019-10-16_H6_groundtruth.csv /Users/maggie/Desktop/HDstorage/H6/H6_GROUNDTRUTH/2019-10-16_H6_groundtruth.csv
/Volumes/TOSHIBA-18/H6-black/GroundTruth/GroundTruth/2019-10-17_H6_groundtruth.csv /Users/maggie/Desktop/HDstorage/H6/H6_GROUNDTRUTH/2019-10-17_H6_groundtruth.csv
/Volumes/TOSHIBA-18

### 2. Compress all days

In [13]:
to_compress = os.path.join(local_save_path, home, f'{home}_GROUNDTRUTH')
destination = os.path.join(dest_for_compressed, home, os.path.basename(to_compress.strip('/')))

print(f'from: {to_compress} to: {destination}')
shutil.make_archive(destination, 'zip', to_compress)


from: /Users/maggie/Desktop/HDstorage/H6/H6_GROUNDTRUTH to: /Volumes/SAMSUNG-4/H6/H6_GROUNDTRUTH


'/Volumes/SAMSUNG-4/H6/H6_GROUNDTRUTH.zip'

***
### H2 - dark image investigation

In [ ]:
import pandas as pd

In [32]:
summaries = glob('/Volumes/TOSHIBA-22/H2-red/Summaries/*-img-summary.txt')
print(summaries)

img_csv_summary = '/Users/maggie/Desktop/image_df_counts.csv'

summary_df = pd.read_csv(img_csv_summary, index_col='Unnamed: 0')

hub_dfs = []
for txtfile in summaries:
    txt_df = pd.read_csv(txtfile, delimiter=' ', usecols=['day', '%Capt', '%Dark'], index_col='day')
#     print(txt_df.columns)
    hub = os.path.basename(txtfile).split('-')[1]
#     print(hub)
    col_names = [f'{hub}_Capt', f'{hub}_Dark']
    txt_df.columns = col_names
    hub_dfs.append(txt_df)

hub_dfs.append(summary_df)
all_df = pd.concat(hub_dfs, axis=1)
all_df = all_df.reindex(sorted(all_df.columns), axis=1)
all_df = all_df.sort_index(axis=0)
print(all_df)

all_df.to_csv('/Users/maggie/Desktop/H2_all_images_summary.csv')
    
    

['/Volumes/TOSHIBA-22/H2-red/Summaries/H2-RS1-img-summary.txt', '/Volumes/TOSHIBA-22/H2-red/Summaries/H2-RS2-img-summary.txt', '/Volumes/TOSHIBA-22/H2-red/Summaries/H2-RS4-img-summary.txt', '/Volumes/TOSHIBA-22/H2-red/Summaries/H2-RS5-img-summary.txt']
            RS1_Capt  RS1_Dark  RS1_Img  RS2_Capt  RS2_Dark  RS2_Img  \
2019-03-01       NaN       NaN      NaN      0.27      0.00      NaN   
2019-03-02       NaN       NaN      NaN      1.00      0.21      NaN   
2019-03-03       NaN       NaN      NaN      1.00      0.09      NaN   
2019-03-04       NaN       NaN      NaN      1.00      0.48      NaN   
2019-03-05       NaN       NaN      NaN      1.00      0.25      NaN   
2019-03-06       NaN       NaN      NaN      1.00      0.25      NaN   
2019-03-07       NaN       NaN      NaN      1.00      0.20      NaN   
2019-03-08       NaN       NaN      NaN      1.00      0.00      NaN   
2019-03-09       NaN       NaN      NaN      1.00      0.21      NaN   
2019-03-10       NaN       